# Intro

When I started writing [Python](https://www.python.org/) some eight or nine years ago one of the first libraries I was introduced to was
[`scikit-learn`](https://scikit-learn.org/stable/).
Being a beginner, I had to learn how to "read the docs," which can be challenging for anyone exploring unfamiliar software.
Articles, videos, doc strings, example code snippets, source code, and even unit tests.
I see them all as another side to the same box.
When one didn't make sense, I could search for another to help me better understand the puzzle.

The more I studied `scikit-learn`, the more I'd try to rebuild examples using techniques I was comfortable with:
[`pandas`](https://pandas.pydata.org/), [`numpy`](https://numpy.org/), hand-written math and sketches.
Fairly quickly I noticed the
[`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
was used in a lot of code
[examples](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#examples-using-sklearn-pipeline-pipeline).
The idea of a pipeline wasn't new—the output of one operation was the input to next—but knowing how to read and use one was.
Once I figured it out,
the `Pipeline` earned a place in my toolbox and serves as a part of my signature style when working in machine learning.

I now find it much easier to use the [`scikit-learn` API](https://scikit-learn.org/stable/modules/classes.html),
as well as [contribute](https://github.com/scikit-learn/scikit-learn/pull/25220)
to its [source code](https://github.com/scikit-learn/scikit-learn).
With that, I'd like to share some alternative approaches to building a machine learning model using `scikit-learn`'s `Pipeline` and friends:
[`FeatureUnion`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html),
[`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html),
and [`FunctionTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html).

# Setup

I'll start with the prerequisites;
to guarantee that you get the same results as I do I'd suggest installing the following package versions.
You can do that by uncommenting the below cell and running it.

In [1]:
# !pip install numpy==1.24.3
# !pip install pandas==2.1.4
# !pip install scikit-learn==1.3.2

Below are the initial imports.
I'll introduce more in each section as we go.
The comments should describe what each is going to be used for,
but if you have questions drop a comment at the bottom of the post.

In [2]:
import warnings  # To suppress some warnings.

import numpy as np  # For numerical computation when a dataframe isn't available.
import pandas as pd  # For reading/manipulating data.
from sklearn.impute import SimpleImputer  # For imputing missing values.
from sklearn.linear_model import LogisticRegression  # Simple classifier.
from sklearn.model_selection import train_test_split  # Split train data into train/val.
from sklearn.preprocessing import MinMaxScaler  # Simple preprocessing step.

# Filtering out a scikit-learn warning related to the `LogisticRegression` model
# not converging in the last section. This does not pertain to the tutorial so it
# will be hidden.
warnings.filterwarnings(action="ignore", module="sklearn")

I'm using the Titanic data set from [Kaggle](https://www.kaggle.com/).
You can download it from [here](https://www.kaggle.com/competitions/titanic/data).

The goal of the competition is to build a classification model that can correctly predict if a passenger survived the Titanic.
I'm not aiming for a state-of-the-art model here;
I'm sharing how one might build a model and then introduce the `Pipeline` et al.

In [3]:
# Read the Titanic data set.
train = pd.read_csv("train.csv", index_col="PassengerId")
test = pd.read_csv("test.csv", index_col="PassengerId")

# Separate X and y.
X = train.drop(columns="Survived")
y = train.Survived

# Split into train/val data.
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, stratify=y)

# Multi-step Transformation

Let's start off simple.
Suppose we want to use the continuous features to predict survival.

In [4]:
# Limit to features with dtype float.
cont_cols = X_train.select_dtypes(include="float").columns
X_train_float = X_train[cont_cols]
X_val_float = X_val[cont_cols]

We could scale the features between $[0, 1]$ using
[`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).

In [5]:
# Scale data.
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_float)
X_val_scaled = scaler.transform(X_val_float)

Use the [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)
to fill any missing values with the average across all passengers.

In [6]:
# Impute data.
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train_scaled)
X_val_imputed = imputer.transform(X_val_scaled)

And fit our
[`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
with our scaled-then-imputed data and check out the score on our held-out data.

In [7]:
# Fit the model and grade against val data.
clf = LogisticRegression(random_state=0)
clf.fit(X_train_imputed, y_train)
clf.score(X_val_imputed, y_val)

0.6457399103139013

It's a straightforward and easy-to-understand series of steps.
1. Scale
2. Impute
3. Fit

# [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

Here's how we'd do it with a `Pipeline`.

In [8]:
# Import Pipeline.
from sklearn.pipeline import Pipeline

In [9]:
# Limit to features with dtype float.
cont_cols = X_train.select_dtypes(include="float").columns
X_train_float = X_train[cont_cols]
X_val_float = X_val[cont_cols]

In [10]:
# Define a pipe with three steps: scale -> impute -> classify.
pipe = Pipeline(
    steps=[
        ("scale", MinMaxScaler()),
        ("impute", SimpleImputer()),
        ("clf", LogisticRegression(random_state=0)),
    ],
)
pipe.fit(X_train_float, y_train)
pipe.score(X_val_float, y_val)

0.6457399103139013

In 10 lines of code (with whitespace) we created a process to take data as input, scale it, impute it, and classify it.
Here's what our `pipe` looks like.

In [11]:
# HTML representation of the pipe.
pipe

Pipeline(steps=[('scale', MinMaxScaler()), ('impute', SimpleImputer()),
                ('clf', LogisticRegression(random_state=0))])

Compared to the previous method, the `pipe` keeps everything in one composable object.
We don't have to store each transformer or estimator as its own variable that we may or may not forget later.
And we don't have to store the output after each transformation, which includes both the training and validation data.
All together we create four variables: `cont_cols`, `X_train_float`, `X_val_float`, and `pipe`.
The previous method has ten:
`cont_cols`, `X_train_float`, `X_val_float`,
`scaler`, `X_train_scaled`, `X_val_scaled`,
`imputer`, `X_train_imputed`, `X_val_imputed`,
and `clf`.

# Expanding Feature Space

What if we wanted to transform features in not one, but two ways?
First lets use the
[`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
to transform some categorical/ordinal features.
Then we'll independently use the
[`TargetEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.TargetEncoder.html)
to transform those same features.

In [12]:
# Import one-hot and target encoding transformers
from sklearn.preprocessing import OneHotEncoder, TargetEncoder

We will consider features with less than 10 unique values to be categorical/ordinal.

In [13]:
# Get the number of unique values in each feature sorted ascendingly.
X_train.nunique().sort_values()

Sex           2
Pclass        3
Embarked      3
SibSp         7
Parch         7
Age          82
Cabin       120
Fare        216
Ticket      537
Name        668
dtype: int64

In [14]:
# Limit to categorical/ordinal features.
cat_ord_cols = ["Sex", "Pclass", "Embarked", "SibSp", "Parch"]
X_train_enc = X_train[cat_ord_cols]
X_val_enc = X_val[cat_ord_cols]

We one-hot-encode the features, dropping the first and only tracking the five most frequent values.
This helps us to avoid "exploding" our feature space.

In [15]:
# One-hot-encode categorical/ordinal features.
ohe = OneHotEncoder(drop="first", sparse_output=False, max_categories=5)
X_train_ohe = ohe.fit_transform(X_train_enc)
X_val_ohe = ohe.transform(X_val_enc)

Target encoding only transforms the values of each feature, so we don't have to worry about the number of columns growing.

In [16]:
# Target encode categorical/ordinal features.
tgt = TargetEncoder(random_state=0)
X_train_tgt = tgt.fit_transform(X_train_enc, y_train)
X_val_tgt = tgt.transform(X_val_enc)

We stack our two transformed data sets along the column-axis,
keeping our row count the same but increasing the number of columns.

In [17]:
# Join One-hot-encoded features with target-encoded features.
X_train_feat_union = np.hstack((X_train_ohe, X_train_tgt))
X_val_feat_union = np.hstack((X_val_ohe, X_val_tgt))

In [18]:
clf.fit(X_train_feat_union, y_train)
clf.score(X_val_feat_union, y_val)

0.7847533632286996

It looks like there's some signal coming from the `cat_ord_cols`!

# [`FeatureUnion`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html)

In [19]:
# Import FeatureUnion
from sklearn.pipeline import FeatureUnion

In [20]:
# Define a feature union with two transformers: ohe & tgt.
cat_ord = FeatureUnion(
    transformer_list=[
        ("ohe", OneHotEncoder(drop="first", sparse_output=False, max_categories=5)),
        ("tgt", TargetEncoder(random_state=0)),
    ],
)
_train = cat_ord.fit_transform(X_train_enc, y_train)
_val = cat_ord.transform(X_val_enc)

In [21]:
np.allclose(a=X_train_feat_union, b=_train)

True

In [22]:
np.allclose(a=X_val_feat_union, b=_val)

True

In [23]:
pipe = Pipeline(
    steps=[
        ("cat_ord", cat_ord),
        ("clf", LogisticRegression(random_state=0)),
    ],
)
pipe.fit(X_train_enc, y_train)
pipe.score(X_val_enc, y_val)

0.7847533632286996

In [24]:
pipe

Pipeline(steps=[('cat_ord',
                 FeatureUnion(transformer_list=[('ohe',
                                                 OneHotEncoder(drop='first',
                                                               max_categories=5,
                                                               sparse_output=False)),
                                                ('tgt',
                                                 TargetEncoder(random_state=0))])),
                ('clf', LogisticRegression(random_state=0))])

# Transform Feature Subsets

In [25]:
# Limit to features with dtype float.
cont_cols = X_train.select_dtypes(include="float").columns
X_train_float = X_train[cont_cols]
X_val_float = X_val[cont_cols]

# Continuous feature pipeline.
cont_pipe = Pipeline(
    steps=[
        ("scale", MinMaxScaler()),
        ("impute", SimpleImputer()),
    ],
)
X_train_cont = cont_pipe.fit_transform(X_train_float)
X_val_cont = cont_pipe.transform(X_val_float)

In [26]:
# Limit to categorical/ordinal features.
cat_ord_cols = ["Sex", "Pclass", "Embarked", "SibSp", "Parch"]
X_train_enc = X_train[cat_ord_cols]
X_val_enc = X_val[cat_ord_cols]

# Categorical/ordinal feature union.
cat_ord = FeatureUnion(
    transformer_list=[
        ("ohe", OneHotEncoder(drop="first", sparse_output=False, max_categories=5)),
        ("tgt", TargetEncoder(random_state=0)),
    ],
)
X_train_feat_union = cat_ord.fit_transform(X_train_enc, y_train)
X_val_feat_union = cat_ord.transform(X_val_enc)

In [27]:
# Join continuous transformations with categorical/ordinal transformations.
X_train_join = np.hstack((X_train_cont, X_train_feat_union))
X_val_join = np.hstack((X_val_cont, X_val_feat_union))

In [28]:
clf.fit(X_train_join, y_train)
clf.score(X_val_join, y_val)

0.8026905829596412

# [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

In [29]:
from sklearn.compose import ColumnTransformer

In [30]:
col_trf = ColumnTransformer(
    transformers=[
        ("cont_pipe", cont_pipe, cont_cols),
        ("cat_ord", cat_ord, cat_ord_cols),
    ],
    remainder="drop",
)
_train = col_trf.fit_transform(X_train, y_train)
_val = col_trf.transform(X_val)

In [31]:
np.allclose(a=X_train_join, b=_train)

True

In [32]:
np.allclose(a=X_val_join, b=_val)

True

In [33]:
pipe = Pipeline(
    steps=[
        ("col_trf", col_trf),
        ("clf", LogisticRegression(random_state=0)),
    ],
)
pipe.fit(X_train, y_train)
pipe.score(X_val, y_val)

0.8026905829596412

In [34]:
pipe

Pipeline(steps=[('col_trf',
                 ColumnTransformer(transformers=[('cont_pipe',
                                                  Pipeline(steps=[('scale',
                                                                   MinMaxScaler()),
                                                                  ('impute',
                                                                   SimpleImputer())]),
                                                  Index(['Age', 'Fare'], dtype='object')),
                                                 ('cat_ord',
                                                  FeatureUnion(transformer_list=[('ohe',
                                                                                  OneHotEncoder(drop='first',
                                                                                                max_categories=5,
                                                                                                sparse_output=False)),
                                                                                 ('tgt',
                                                                                  TargetEncoder(random_state=0))]),
                                                  ['Sex', 'Pclass', 'Embarked',
                                                   'SibSp', 'Parch'])])),
                ('clf', LogisticRegression(random_state=0))])

# Custom Transformer

In [35]:
X_train.Name.sample(n=5, random_state=0)

PassengerId
44     Laroche, Miss. Simonne Marie Anne Andree
257              Thorne, Mrs. Gertrude Maybelle
232                    Larsson, Mr. Bengt Edvin
213                      Perkin, Mr. John Henry
290                        Connolly, Miss. Kate
Name: Name, dtype: object

In [36]:
import re

def get_title(
    text: str,
    title_pattern: str = r"Mrs?|Miss|Master",
) -> str | None:
    """Get a passenger's title if present.
    
    If more than one title found, return title with
    the least number of characters.
    
    The defalut title_pattern will detect:
    - Mr
    - Mrs
    - Miss
    - Master
    """
    possible_titles: set[str] = set(re.findall(pattern=title_pattern, string=text))
    title: list[str] = sorted(possible_titles, key=len)
    if title:
        return title.pop(0)

In [37]:
# Assert function extracts expected title.
assert get_title("Turpin, Mr. William John Robert") == "Mr"

In [38]:
# Assert function returns nothing if no title present.
assert get_title("Rothes, the Countess. of (Lucy Noel Martha Dyer-Edwards)") is None

In [39]:
# Assert function returns title with least number of characters.
assert get_title("Mr. and Mrs. Smith") == "Mr"

In [40]:
# Vectorize get_title allowing input to be array-like.
# Note that output dtypes will all be the same (None -> "None")
get_title_vec = np.vectorize(get_title)

np.unique(get_title_vec(X_train.Name), return_counts=True)

(array(['Master', 'Miss', 'Mr', 'Mrs', 'None'], dtype='<U6'),
 array([ 33, 133, 384,  98,  20], dtype=int64))

In [41]:
# Get titles from names.
X_train_title = X_train.assign(Title=get_title_vec(X_train.Name))
X_val_title = X_val.assign(Title=get_title_vec(X_val.Name))

In [42]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer()

In [43]:
# One-hot-encode title and join with age.
age_trf = ColumnTransformer(
    transformers=[
        ("age", "passthrough", ["Age"]),
        ("ohe", OneHotEncoder(drop=["None"], sparse_output=False), ["Title"]),
    ],
    remainder="drop",
)

# Impute missing values (age) using title and age
knn_impute_pipe = Pipeline(
    steps=[
        ("age_trf", age_trf),
        ("knn_impute", KNNImputer()),
    ],
).set_output(transform="pandas")

In [44]:
# Get sample of passengers with missing ages.
title_missing_age_sample = X_train_title.loc[
    X_train_title.Age.isnull()
].groupby("Title").apply(lambda f: f.index[0])

# Compare imputed age using mean with KNN.
imputer = SimpleImputer().set_output(transform="pandas")
_mean = imputer.fit_transform(X_train_title[["Age"]]).loc[title_missing_age_sample]
_knn = knn_impute_pipe.fit_transform(X_train_title).loc[title_missing_age_sample]
_impute = pd.concat([_mean, _knn], keys=["Mean", "KNN"], axis="columns")
_impute

Mean      KNN                                    \
                   Age age__Age ohe__Title_Master ohe__Title_Miss   
PassengerId                                                         
177          29.796842      4.8               1.0             0.0   
33           29.796842     20.0               0.0             1.0   
251          29.796842     26.4               0.0             0.0   
579          29.796842     35.6               0.0             0.0   
767          29.796842     40.8               0.0             0.0   

                                          
            ohe__Title_Mr ohe__Title_Mrs  
PassengerId                               
177                   0.0            0.0  
33                    0.0            0.0  
251                   1.0            0.0  
579                   0.0            1.0  
767                   0.0            0.0

# [`FunctionTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html)

In [45]:
from sklearn.preprocessing import FunctionTransformer

In [46]:
# Convert get_title_vec into an sklearn transformer
title_func = FunctionTransformer(func=get_title_vec)

In [47]:
title_func

FunctionTransformer(func=<numpy.vectorize object at 0x000001E3F8F4B650>)

In [48]:
# Pipeline to get titles, then one-hot-encode.
title_pipe = Pipeline(
    steps=[
        ("title_func", title_func),
        ("ohe", OneHotEncoder(drop=["None"], sparse_output=False)),
    ],
)

In [49]:
title_pipe

Pipeline(steps=[('title_func',
                 FunctionTransformer(func=<numpy.vectorize object at 0x000001E3F8F4B650>)),
                ('ohe', OneHotEncoder(drop=['None'], sparse_output=False))])

In [50]:
# Pipeline to get titles and passthrough age.
age_title_trf = ColumnTransformer(
    transformers=[
        ("title_pipe", title_pipe, ["Name"]),
        ("age", "passthrough", ["Age"])
    ],
    remainder="drop",
)

In [51]:
age_title_trf

ColumnTransformer(transformers=[('title_pipe',
                                 Pipeline(steps=[('title_func',
                                                  FunctionTransformer(func=<numpy.vectorize object at 0x000001E3F8F4B650>)),
                                                 ('ohe',
                                                  OneHotEncoder(drop=['None'],
                                                                sparse_output=False))]),
                                 ['Name']),
                                ('age', 'passthrough', ['Age'])])

In [52]:
# Pipeline to impute age given ages of neighbors with given titles.
age_pipe = Pipeline(
    steps=[
        ("age_title_trf", age_title_trf),
        ("impute_knn", KNNImputer()),
    ],
)

In [53]:
age_pipe

Pipeline(steps=[('age_title_trf',
                 ColumnTransformer(transformers=[('title_pipe',
                                                  Pipeline(steps=[('title_func',
                                                                   FunctionTransformer(func=<numpy.vectorize object at 0x000001E3F8F4B650>)),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop=['None'],
                                                                                 sparse_output=False))]),
                                                  ['Name']),
                                                 ('age', 'passthrough',
                                                  ['Age'])])),
                ('impute_knn', KNNImputer())])

In [54]:
col_trf = ColumnTransformer(
    transformers=[
        ("fare", cont_pipe, ["Fare"]),
        ("age", age_pipe, ["Age", "Name"]),
        ("cat_ord", cat_ord, cat_ord_cols),
    ],
    remainder="drop",
)

In [55]:
col_trf

ColumnTransformer(transformers=[('fare',
                                 Pipeline(steps=[('scale', MinMaxScaler()),
                                                 ('impute', SimpleImputer())]),
                                 ['Fare']),
                                ('age',
                                 Pipeline(steps=[('age_title_trf',
                                                  ColumnTransformer(transformers=[('title_pipe',
                                                                                   Pipeline(steps=[('title_func',
                                                                                                    FunctionTransformer(func=<numpy.vectorize object at 0x000001E3F8F4B650>)),
                                                                                                   ('ohe',
                                                                                                    OneHotEncoder(drop=['None'],
                                                                                                                  sparse_output=False))]),
                                                                                   ['Name']),
                                                                                  ('age',
                                                                                   'passthrough',
                                                                                   ['Age'])])),
                                                 ('impute_knn', KNNImputer())]),
                                 ['Age', 'Name']),
                                ('cat_ord',
                                 FeatureUnion(transformer_list=[('ohe',
                                                                 OneHotEncoder(drop='first',
                                                                               max_categories=5,
                                                                               sparse_output=False)),
                                                                ('tgt',
                                                                 TargetEncoder(random_state=0))]),
                                 ['Sex', 'Pclass', 'Embarked', 'SibSp',
                                  'Parch'])])

In [56]:
pipe = Pipeline(
    steps=[
        ("col_trf", col_trf),
        ("clf", LogisticRegression(random_state=0)),
    ],
)

In [57]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('col_trf',
                 ColumnTransformer(transformers=[('fare',
                                                  Pipeline(steps=[('scale',
                                                                   MinMaxScaler()),
                                                                  ('impute',
                                                                   SimpleImputer())]),
                                                  ['Fare']),
                                                 ('age',
                                                  Pipeline(steps=[('age_title_trf',
                                                                   ColumnTransformer(transformers=[('title_pipe',
                                                                                                    Pipeline(steps=[('title_func',
                                                                                                                     FunctionTransformer(func=<numpy.vectorize object at 0x000001E3F8F4B650>)),
                                                                                                                    ('ohe',
                                                                                                                     OneHotEnc...
                                                                                                                                   sparse_output=False))]),
                                                                                                    ['Name']),
                                                                                                   ('age',
                                                                                                    'passthrough',
                                                                                                    ['Age'])])),
                                                                  ('impute_knn',
                                                                   KNNImputer())]),
                                                  ['Age', 'Name']),
                                                 ('cat_ord',
                                                  FeatureUnion(transformer_list=[('ohe',
                                                                                  OneHotEncoder(drop='first',
                                                                                                max_categories=5,
                                                                                                sparse_output=False)),
                                                                                 ('tgt',
                                                                                  TargetEncoder(random_state=0))]),
                                                  ['Sex', 'Pclass', 'Embarked',
                                                   'SibSp', 'Parch'])])),
                ('clf', LogisticRegression(random_state=0))])

In [58]:
pipe.score(X_val, y_val)

0.8071748878923767

# Conclusion